In [2]:
from LandBase import *

# 근로소득세납부이력을 DB에 저장 (출처: 국가통계포털)
def tax_history_into_db(p_con, p_yyyy, p_ignore_flag=True) :
    table_name = 'tax_history'
    exist_flag = False

    # 기존 데이터를 무시하지 말라(p_ignore_flag==False)는 옵션이 주어지면
    # 같은 데이터가 이미 있는지 검사해서 있으면 함수를 종료함
    if (p_ignore_flag==False) :
        sql = 'select 1 as cnt from '+table_name +' where tax_yyyy = \'' + p_yyyy + '\' limit 1'
        exist_flag = check_data_exists(p_con, sql)
        if (exist_flag) :
            return
    
    # Open API 호출 (kosis.kr)
    url = 'http://kosis.kr/openapi/statisticsData.do?method=getList&apiKey=' + kosis_kr_apikey \
    + '&format=json&jsonVD=Y' \
    + '&userStatsId=yyj1219/133/DT_13301_013/2/1/20170915101721&prdSe=Y' \
    + '&startPrdDe=' + p_yyyy \
    + '&endPrdDe=' + p_yyyy
    
    # json 형식으로 반환된 Open API 결과를 DataFrame으로 변환 (불필요한 항목은 빼고, 근로소득세만 남김)
    response = requests.get(url)
    df = pd.read_json(response.text)
    df.drop(['C1', 'C1_NM_ENG', 'C1_OBJ_NM', 'C1_OBJ_NM_ENG', 'C2', 'C2_NM_ENG', 'C2_OBJ_NM', 'C2_OBJ_NM_ENG', \
            'ITM_ID', 'ITM_NM', 'ITM_NM_ENG', 'ORG_ID', 'PRD_SE', \
            'TBL_ID', 'TBL_NM', 'UNIT_NM', 'UNIT_NM_ENG'], axis=1, inplace=True)
    #df = df[ (df['C1_NM']=='근로소득세') & (df['DT']!='-') ]
    df = df[ (df['DT']!='-') ]
    df.rename(columns={'PRD_DE':'tax_yyyy', 'C1_NM':'tax_type', 'C2_NM':'tax_office', 'DT':'tax_amount'}, inplace=True)

    # DB 처리
    con.execute('delete from ' + table_name + ' where tax_yyyy = \'' + p_yyyy + '\'')
    df.to_sql(name=table_name, con=p_con,  if_exists='append', index=False, chunksize=1000)
    con.execute('commit')


In [ ]:
# DB Connection Open
con = get_db_connection()

# 세금납부이력을 DB에 저장
# 테이블: tax_history
yyyy_list = get_yyyy_list('2012', '2016')
for yyyy in yyyy_list :
    tax_history_into_db(p_con=con, p_yyyy=yyyy, p_ignore_flag=False) 

# DB Connection Close
disconnect_db(con)

print('Program Finised')